## Data Integration and Reshaping

environment: pyhton 3.8.8

### Table of Content

### 1 Extrac data:

the xml file is missing root, & should be changed to ''&''

In [3]:
# read the xml file
my_xml = []
with open('realstate.xml') as f:
    for line in f:
        # fix the text issue
        fix_line = line.replace('&','&amp;')
        my_xml.append(fix_line)
    f.close()
# create a new file and add root
with open('fixed_realstate.xml','a') as f:
    f.write("<properties>\n")
    for line in my_xml:
        string = '    ' + line
        f.write(string)
    f.write("</properties>\n")
    f.close()

Parse xml file

In [4]:
import xml.etree.ElementTree as ET
import pandas as pd

xml_data = open('fixed_realstate.xml', 'r').read() 
root = ET.XML(xml_data)
# list for dataframe attributes
property_id  = []
lat  = []
lng =[]
addr_street = []
for i, attribute  in enumerate(root):
    # match the attribute and store information
    for info in attribute:
        if  info.tag == 'property_id':
            property_id.append(info.text)
        elif info.tag == 'lat':
            lat.append(info.text)
        elif info.tag == 'lng':
            lng.append(info.text)
        elif info.tag == 'addr_street':
            addr_street.append(info.text)
        else:
            # in case, If there are any strange errors
            print(info.tag,info.text)
# convert to pandas dataframe
data = [property_id,lat,lng,addr_street]       
address_df = pd.DataFrame(data).T 
cols = ['property_id','lat','lng','addr_street']
address_df.columns = cols

In [5]:
address_df.head()

,property_id,lat,lng,addr_street
0,73532,-37.849075,145.303972,31 Dorrigo Drive
1,2194,-37.819211,144.845848,35 Corrigan Avenue
2,12980,-37.694174,144.758153,32 Dundee Way
3,7687,-37.74916077,144.7638855,95 Fairfax Circuit
4,3210,-37.814997,144.86763100000005,25A Sanderson Street


Read the pdf file

In [6]:
import PyPDF2
# creating a pdf file object
pdfFileObj = open('realstate.pdf', 'rb')
  
# creating a pdf reader object
pdfReader  = PyPDF2.PdfReader(pdfFileObj)

pages_text = []
 
# creating a page object
pageObj = pdfReader.pages[0]
pages_text.append(pageObj.extract_text())

#convert to pandas df
df_pdf = pd.DataFrame({'Text': pages_text})

pdfFileObj.close()

In [7]:
from tabula.io import read_pdf
tables = read_pdf('realstate.pdf', pages='all')

In [8]:
df_pdf = tables[0]
df_pdf = df_pdf.drop('property_id', axis=1)
rename = {'Unnamed: 0': 'property_id'}
df_pdf.rename(columns=rename,
          inplace=True)
df_pdf = df_pdf[['property_id','lat','lng','addr_street']]
df_pdf.head()

,property_id,lat,lng,addr_street
0,34701,-37.722324,145.021654,33 Boldrewood Parade
1,12218,-37.712146,144.771964,12 Lacebark Road
2,95829,-37.925040,145.047606,46 London Street
3,37225,-37.645165,145.052382,3 Guinea Court
4,94289,-37.960549,145.078754,14 Oakmont Crescent


In [9]:
# concat both df
frames = [df_pdf, address_df]
address_df = pd.concat(frames)
address_df.reset_index(inplace=True, drop=True) 

In [10]:
address_df.head()

,property_id,lat,lng,addr_street
0,34701,-37.722324,145.021654,33 Boldrewood Parade
1,12218,-37.712146,144.771964,12 Lacebark Road
2,95829,-37.92504,145.047606,46 London Street
3,37225,-37.645165,145.052382,3 Guinea Court
4,94289,-37.960549,145.078754,14 Oakmont Crescent


### 2.Data Transformation:

In [11]:
address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  2191 non-null   object
 1   lat          2191 non-null   object
 2   lng          2191 non-null   object
 3   addr_street  2191 non-null   object
dtypes: object(4)
memory usage: 68.6+ KB


In [12]:
#change the data type of lat and lng to float
address_df['lat'] = address_df['lat'].astype(float)
address_df['lng'] = address_df['lng'] .astype(float)
address_df['property_id'] = address_df['property_id'].astype(int)
address_df['addr_street'] = address_df['addr_street'].astype(str)

In [13]:
# remove duplicated values

address_df.sort_values("property_id", inplace=True)
address_df.drop_duplicates(subset="property_id",
                     keep=False, inplace=True)
address_df.reset_index(inplace=True, drop=True) 
address_df.head()

,property_id,lat,lng,addr_street
0,70,-37.824362,144.962089,117/173 City Road
1,271,-37.807817,144.897262,12 Pole Street
2,313,-37.806835,144.901581,13 Parker Street
3,402,-37.797932,144.895828,16 Malden Street
4,403,-37.797905,144.895897,16 Malden Street


In [14]:
# add columns to dataframe and set defualt values
address_df['suburb'] = 'not available'
address_df['closest_train_station_id'] = 0
address_df['distance_to_closest_train_station'] = 0
address_df['travel_min_to_MC'] = -1
address_df['direct_journey_flag'] = -1
address_df['Type'] = 'not identified'


In [15]:
# read shapefile and extract suburb name and it's boundary coordinate
import shapefile as shp


shape = shp.Reader("./vic_suburb_bounadry/VIC_LOCALITY_POLYGON_shp.shp")
#shape.shapeRecords()[0].shape.__geo_interface__
#shape.records()
suburb_name = []
suburb_coordinates = []
for record in range(len(shape)):
    coordinate = shape.shapeRecords()[record].shape.__geo_interface__
    suburb_name.append(shape.records()[record][6])
    suburb_coordinates.append(coordinate['coordinates'])
suburb_df = pd.DataFrame([suburb_name,suburb_coordinates]).T 
cols = ['suburb_name','suburb_coordinates']
suburb_df.columns = cols
suburb_df.head()

,suburb_name,suburb_coordinates
0,UNDERBOOL,"[[(141.74552399, -35.07228701), (141.74552471,..."
1,NURRAN,"[[(148.668767, -37.39571245), (148.66876202, -..."
2,WOORNDOO,"[[(142.92287999, -37.97885997), (142.90449196,..."
3,DEPTFORD,"[[(147.82335712, -37.66000897), (147.8231274, ..."
4,YANAC,"[[(141.279783, -35.99858911), (141.27988533, -..."


In [16]:
# remove duplicated values

suburb_df.sort_values("suburb_name", inplace=True)
suburb_df.drop_duplicates(subset="suburb_name",
                     keep=False, inplace=True)
suburb_df.reset_index(drop=True, inplace=True)

In [17]:
import matplotlib.path as mplPath

#assign suburb to address
for x in range(len(address_df)):
    # extract the coordinate of the address
    coordinate = [address_df['lng'][x],address_df['lat'][x]]
    for i in range(len(suburb_df)):
        #use mplPath.Path function to find the suburb
        poly_path = mplPath.Path(suburb_df['suburb_coordinates'][i][0])
        if poly_path.contains_point(coordinate):
            address_df.at[x,'suburb'] = suburb_df['suburb_name'][i]
            break

In [18]:
from math import sin, cos, sqrt, atan2, radians, log
# function for  haversine_distances
def haversine_distances(lati1,long1,lati2,long2):
    
    R = 6378.0 #radius of the earth.

    lat1 = radians(lati1)
    lon1 = radians(long1)
    lat2 = radians(lati2)
    lon2 = radians(long2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2.0)**2.0 + cos(lat1) * cos(lat2) * sin(dlon / 2.0)**2.0
    c = 2.0 * atan2(sqrt(a), sqrt(1.0 - a))

    distance = R * c
    return distance

In [19]:
station_stops  = pd.read_csv('./metropolitan/stops.txt')

In [20]:
# change the datatype of distance_to_closest_train_station to float
address_df['distance_to_closest_train_station'] = pd.to_numeric(address_df['distance_to_closest_train_station'], downcast="float")
for row in range(len(address_df)):
    # extract coordinates of the property and station
    distances = []
    lati1= address_df['lat'][row]
    long1= address_df['lng'][row]
    #calculate the distance to all station
    for station in range(len(station_stops)):
        lati2= station_stops['stop_lat'][station]
        long2= station_stops['stop_lon'][station]
        distance = haversine_distances(lati1,long1,lati2,long2)
        distances.append(distance)
    #find the minimum distance and the closet station
    address_df.at[row,'closest_train_station_id'] = station_stops['stop_id'][distances.index(min(distances))]
    address_df.at[row,'distance_to_closest_train_station'] = min(distances)
address_df.head()

,property_id,lat,lng,addr_street,suburb,closest_train_station_id,distance_to_closest_train_station,travel_min_to_MC,direct_journey_flag,Type
0,70,-37.824362,144.962089,117/173 City Road,SOUTHBANK,19854,0.798983,-1,-1,not identified
1,271,-37.807817,144.897262,12 Pole Street,SEDDON,19997,0.192504,-1,-1,not identified
2,313,-37.806835,144.901581,13 Parker Street,FOOTSCRAY,19997,0.573100,-1,-1,not identified
3,402,-37.797932,144.895828,16 Malden Street,FOOTSCRAY,20024,0.637143,-1,-1,not identified
4,403,-37.797905,144.895897,16 Malden Street,FOOTSCRAY,20024,0.643156,-1,-1,not identified


In [21]:
# extract Melbourne central station's id
mc_id = station_stops.at[station_stops[station_stops['stop_name']== 'Melbourne Central Railway Station'].index[0], 'stop_id']

trips  = pd.read_csv('./metropolitan/trips.txt')

Calculate travel_min_to_MC and direct_journey_flag

In [23]:
#load data
service = pd.read_csv('./metropolitan/calendar.txt')
stop_times  = pd.read_csv('./metropolitan/stop_times.txt')
#extract weekday service id
weekday = service[(service['saturday']!=1) & (service['sunday']!=1) ]['service_id'].tolist()
extract1  = trips[['trip_id','service_id']]
# assign trip id to trip by merge trips and stop_times
stop_times = pd.merge(extract1, stop_times, on='trip_id')
stop_times.head()

,trip_id,service_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,17067982.T0.2-ALM-F-mjp-1.1.H,T0,05:01:00,05:01:00,19847,1,NaN,0,0,0.000000
1,17067982.T0.2-ALM-F-mjp-1.1.H,T0,05:02:00,05:02:00,19848,2,NaN,0,0,723.017818
2,17067982.T0.2-ALM-F-mjp-1.1.H,T0,05:04:00,05:04:00,19849,3,NaN,0,0,1951.735072
3,17067982.T0.2-ALM-F-mjp-1.1.H,T0,05:06:00,05:06:00,19850,4,NaN,0,0,2899.073349
4,17067982.T0.2-ALM-F-mjp-1.1.H,T0,05:08:00,05:08:00,19851,5,NaN,0,0,3927.090952


In [24]:
from datetime import datetime

# for calculated stations
calculated_list = []
for i in range(len(address_df)):
    # get the closet station id
    station = address_df['closest_train_station_id'][i]
    #list to store trip time
    trip_time = []
    # if this station is not calculated 
    if station not in calculated_list and station !=mc_id:
        calculated_list.append(station)
        #get a trip list for the closet station when the depature time is 7-9am and during weekdays
        valid_trips = stop_times['trip_id'].loc[(stop_times['stop_id'] == station) & (stop_times['service_id'].isin(weekday)) &
                      (stop_times['departure_time']<'09:00:00')& (stop_times['departure_time']> '07:00:00')].unique().tolist()
        for trip in range(len(valid_trips)):
            #get a dataframe for this trip and extract melbourne central station or closet station 
            data = stop_times.loc[(stop_times['trip_id'] == valid_trips[trip]) 
                                  & ((stop_times['stop_id'] == station) | (stop_times['stop_id'] == mc_id)) ]
            #if we find both stations
            if len(data) == 2:
                #make sure melbourne central is the destination by compare the stop sequence
                if data[data['stop_id'] == station]['stop_sequence'].values[0] < data[data['stop_id'] == mc_id]['stop_sequence'].values[0]:
                    #calculate travel time
                    time_1 = datetime.strptime(data[data['stop_id'] == station]['departure_time'].values[0],"%H:%M:%S")
                    time_2 = datetime.strptime(data[data['stop_id'] == mc_id]['arrival_time'].values[0],"%H:%M:%S")
                    time = ((time_2 - time_1).seconds/60)
                    trip_time.append(time)
        #all trips are calculated now 
        #get average value
        #means no direct journey or no journey 
        if len(trip_time) == 0:
            ave_time = 'Not available'
        else:
            ave_time = int(sum(trip_time)/len(trip_time))
        #same station should have same travel_min_to_MC value
        address_df.loc[address_df['closest_train_station_id'] == station, 'travel_min_to_MC'] = ave_time
# if the closet station is melbourne centre
address_df.loc[address_df['closest_train_station_id'] == mc_id, 'travel_min_to_MC'] = 0
#if it is not available
address_df.loc[address_df['travel_min_to_MC'] == -1, 'travel_min_to_MC'] = 'not available'

# if there is a direct journey, then travel_min_to_MC is not 'not available'
address_df.loc[address_df['travel_min_to_MC'] != 'Not available', 'direct_journey_flag'] = 1
# if there is no direct trip, then travel_min_to_MC is not available
address_df.loc[address_df['travel_min_to_MC'] == 'Not available', 'direct_journey_flag'] = 0
address_df.head()

,property_id,lat,lng,addr_street,suburb,closest_train_station_id,distance_to_closest_train_station,travel_min_to_MC,direct_journey_flag,Type
0,70,-37.824362,144.962089,117/173 City Road,SOUTHBANK,19854,0.798983,7,1,not identified
1,271,-37.807817,144.897262,12 Pole Street,SEDDON,19997,0.192504,Not available,0,not identified
2,313,-37.806835,144.901581,13 Parker Street,FOOTSCRAY,19997,0.573100,Not available,0,not identified
3,402,-37.797932,144.895828,16 Malden Street,FOOTSCRAY,20024,0.637143,12,1,not identified
4,403,-37.797905,144.895897,16 Malden Street,FOOTSCRAY,20024,0.643156,12,1,not identified


In [39]:
# identify the type of the property
address_df['Type'] = address_df['Type'].astype(str)
for i in range(len(address_df)):
    if '/' in address_df.iloc[i]['addr_street']:
        address_df.at[i,'Type']= 'Unit'
    else:
        address_df.at[i,'Type'] = 'House'

In [53]:

address_df.head()

,property_id,lat,lng,addr_street,suburb,closest_train_station_id,distance_to_closest_train_station,travel_min_to_MC,direct_journey_flag,Type
0,70.0,-37.824362,144.962089,117/173 City Road,SOUTHBANK,19854.0,0.798983,7,1.0,Unit
1,271.0,-37.807817,144.897262,12 Pole Street,SEDDON,19997.0,0.192504,Not available,0.0,House
2,313.0,-37.806835,144.901581,13 Parker Street,FOOTSCRAY,19997.0,0.573100,Not available,0.0,House
3,402.0,-37.797932,144.895828,16 Malden Street,FOOTSCRAY,20024.0,0.637143,12,1.0,House
4,403.0,-37.797905,144.895897,16 Malden Street,FOOTSCRAY,20024.0,0.643156,12,1.0,House


In [44]:
suburb_df = pd.read_csv('suburb.csv')

In [47]:
vic_df = suburb_df[suburb_df['state']=='vic']

In [54]:
suburb_df.columns

Index(['Unnamed: 0', 'state', 'suburb', 'postcode', 'url',
       'Median_price_house', 'Median_price_unit', 'Quarterly_growth_house',
       'Quarterly_growth_unittwelve_month_growth_house',
       'twelve_month_growth_unit', 'Average_annual_growth_house',
       'Average_annual_growth_unit', 'Weekly_median_rent_house',
       'Weekly_median_rent_unit', 'Gross_rental_yield_house',
       'Gross_rental_yield_unit', 'Number_of_Sales_(12m)_house',
       'Number_of_Sales_(12m)_unit', 'Avg._Days_on_Market_(12m)_house',
       'Avg._Days_on_Market_(12m)_unit', 'Total_population_2011',
       'Total_population_2016', 'Population_change_(5y)_2011',
       'Population_change_(5y)_2016', 'Median_household_income_(p/w)_2011',
       'Median_household_income_(p/w)_2016',
       'Household_income_change_(5y)_2011',
       'Household_income_change_(5y)_2016', 'Median_age_of_persons_2011',
       'Median_age_of_persons_2016', 'DESC_house', 'DESC_unit'],
      dtype='object')

In [56]:
address_df.columns

Index(['property_id', 'lat', 'lng', 'addr_street', 'suburb',
       'closest_train_station_id', 'distance_to_closest_train_station',
       'travel_min_to_MC', 'direct_journey_flag', 'Type'],
      dtype='object')